# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 13 2022 1:52PM,253019,21,418543362,"NBTY Global, Inc.",Released
1,Dec 13 2022 1:46PM,253018,10,Yusen-8898110,Yusen – 3D Matrix,Released
2,Dec 13 2022 1:46PM,253018,10,Yusen-8898112,Yusen – 3D Matrix,Released
3,Dec 13 2022 1:46PM,253018,10,Yusen-8898113,Yusen – 3D Matrix,Released
4,Dec 13 2022 1:46PM,253018,10,Yusen-8898114,Yusen – 3D Matrix,Released
5,Dec 13 2022 1:45PM,253017,19,60017107,"GUSA Granules USA, Inc.",Released
6,Dec 13 2022 1:29PM,253015,19,60017106,"GUSA Granules USA, Inc.",Released
7,Dec 13 2022 1:27PM,253014,10,Enova-11096,Emerginnova,Released
8,Dec 13 2022 1:27PM,253014,10,Enova-11097,Emerginnova,Released
9,Dec 13 2022 1:25PM,253012,10,0086245945,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
48,253014,Released,2
49,253015,Released,1
50,253017,Released,1
51,253018,Released,4
52,253019,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
253014,NaN,NaN,2.0
253015,NaN,NaN,1.0
253017,NaN,NaN,1.0
253018,NaN,NaN,4.0
253019,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
252899,0.0,0.0,1.0
252927,0.0,1.0,4.0
252932,16.0,22.0,42.0
252933,42.0,16.0,5.0
252934,13.0,0.0,7.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
252899,0,0,1
252927,0,1,4
252932,16,22,42
252933,42,16,5
252934,13,0,7


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,252899,0,0,1
1,252927,0,1,4
2,252932,16,22,42
3,252933,42,16,5
4,252934,13,0,7


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,252899,,,1
1,252927,,1,4
2,252932,16,22,42
3,252933,42,16,5
4,252934,13,,7


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 13 2022 1:52PM,253019,21,"NBTY Global, Inc."
1,Dec 13 2022 1:46PM,253018,10,Yusen – 3D Matrix
5,Dec 13 2022 1:45PM,253017,19,"GUSA Granules USA, Inc."
6,Dec 13 2022 1:29PM,253015,19,"GUSA Granules USA, Inc."
7,Dec 13 2022 1:27PM,253014,10,Emerginnova
9,Dec 13 2022 1:25PM,253012,10,ISDIN Corporation
10,Dec 13 2022 1:23PM,253010,10,"Methapharm, Inc."
16,Dec 13 2022 1:18PM,253008,10,"Citieffe, Inc."
17,Dec 13 2022 1:17PM,253007,10,Methapharm-G
19,Dec 13 2022 1:17PM,253007,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Dec 13 2022 1:52PM,253019,21,"NBTY Global, Inc.",,,1
1,Dec 13 2022 1:46PM,253018,10,Yusen – 3D Matrix,,,4
2,Dec 13 2022 1:45PM,253017,19,"GUSA Granules USA, Inc.",,,1
3,Dec 13 2022 1:29PM,253015,19,"GUSA Granules USA, Inc.",,,1
4,Dec 13 2022 1:27PM,253014,10,Emerginnova,,,2
5,Dec 13 2022 1:25PM,253012,10,ISDIN Corporation,,,1
6,Dec 13 2022 1:23PM,253010,10,"Methapharm, Inc.",,,6
7,Dec 13 2022 1:18PM,253008,10,"Citieffe, Inc.",,1,
8,Dec 13 2022 1:17PM,253007,10,Methapharm-G,,1,3
9,Dec 13 2022 1:17PM,253007,10,"Methapharm, Inc.",,1,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 13 2022 1:52PM,253019,21,"NBTY Global, Inc.",1,,
1,Dec 13 2022 1:46PM,253018,10,Yusen – 3D Matrix,4,,
2,Dec 13 2022 1:45PM,253017,19,"GUSA Granules USA, Inc.",1,,
3,Dec 13 2022 1:29PM,253015,19,"GUSA Granules USA, Inc.",1,,
4,Dec 13 2022 1:27PM,253014,10,Emerginnova,2,,
5,Dec 13 2022 1:25PM,253012,10,ISDIN Corporation,1,,
6,Dec 13 2022 1:23PM,253010,10,"Methapharm, Inc.",6,,
7,Dec 13 2022 1:18PM,253008,10,"Citieffe, Inc.",,1,
8,Dec 13 2022 1:17PM,253007,10,Methapharm-G,3,1,
9,Dec 13 2022 1:17PM,253007,10,"Methapharm, Inc.",3,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 13 2022 1:52PM,253019,21,"NBTY Global, Inc.",1,,
1,Dec 13 2022 1:46PM,253018,10,Yusen – 3D Matrix,4,,
2,Dec 13 2022 1:45PM,253017,19,"GUSA Granules USA, Inc.",1,,
3,Dec 13 2022 1:29PM,253015,19,"GUSA Granules USA, Inc.",1,,
4,Dec 13 2022 1:27PM,253014,10,Emerginnova,2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 13 2022 1:52PM,253019,21,"NBTY Global, Inc.",1.0,NaN,NaN
1,Dec 13 2022 1:46PM,253018,10,Yusen – 3D Matrix,4.0,NaN,NaN
2,Dec 13 2022 1:45PM,253017,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
3,Dec 13 2022 1:29PM,253015,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
4,Dec 13 2022 1:27PM,253014,10,Emerginnova,2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 13 2022 1:52PM,253019,21,"NBTY Global, Inc.",1.0,0.0,0.0
1,Dec 13 2022 1:46PM,253018,10,Yusen – 3D Matrix,4.0,0.0,0.0
2,Dec 13 2022 1:45PM,253017,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
3,Dec 13 2022 1:29PM,253015,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
4,Dec 13 2022 1:27PM,253014,10,Emerginnova,2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,6071758,119.0,18.0,0.0
12,505931,4.0,0.0,0.0
15,758799,54.0,38.0,71.0
16,1011821,3.0,1.0,0.0
17,252964,1.0,0.0,0.0
19,506032,2.0,0.0,0.0
20,1011952,11.0,4.0,0.0
21,758990,2.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,6071758,119.0,18.0,0.0
1,12,505931,4.0,0.0,0.0
2,15,758799,54.0,38.0,71.0
3,16,1011821,3.0,1.0,0.0
4,17,252964,1.0,0.0,0.0
5,19,506032,2.0,0.0,0.0
6,20,1011952,11.0,4.0,0.0
7,21,758990,2.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,119.0,18.0,0.0
1,12,4.0,0.0,0.0
2,15,54.0,38.0,71.0
3,16,3.0,1.0,0.0
4,17,1.0,0.0,0.0
5,19,2.0,0.0,0.0
6,20,11.0,4.0,0.0
7,21,2.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,119.0
1,12,Released,4.0
2,15,Released,54.0
3,16,Released,3.0
4,17,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,17,19,20,21
Status,,,,,,,,
Completed,0.0,0.0,71.0,0.0,0.0,0.0,0.0,0.0
Executing,18.0,0.0,38.0,1.0,0.0,0.0,4.0,1.0
Released,119.0,4.0,54.0,3.0,1.0,2.0,11.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,17,19,20,21
0,Completed,0.0,0.0,71.0,0.0,0.0,0.0,0.0,0.0
1,Executing,18.0,0.0,38.0,1.0,0.0,0.0,4.0,1.0
2,Released,119.0,4.0,54.0,3.0,1.0,2.0,11.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,17,19,20,21
0,Completed,0.0,0.0,71.0,0.0,0.0,0.0,0.0,0.0
1,Executing,18.0,0.0,38.0,1.0,0.0,0.0,4.0,1.0
2,Released,119.0,4.0,54.0,3.0,1.0,2.0,11.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()